# [作者 TsukiSama9292 - Github 連結](https://github.com/tsukisama9292)
## [參考文獻 - 連結](https://ianwu.tw/press/topic/command_line_program/yt-dlp.html#%E4%BB%8B%E7%B4%B9)

# 依賴安裝

In [ ]:
!pip install -q yt-dlp
!pip install -q -U gradio

In [ ]:
### 有問題的時候，可以考慮安裝
# !pip install setuptools 

# 使用GUI作為下載工具，預設都是最高畫質和音質

In [ ]:
import gradio as gr
import subprocess
import zipfile
import os
from datetime import datetime

now = datetime.now()
year = now.year
month = now.month
day = now.day
hour = now.hour
minute = now.minute
second = now.second

def download_video(YOUTUBE_URL, do_zip):
    if(do_zip):
        process = subprocess.Popen(
            ['yt-dlp', '-o', 'Download_TMP/%(title)s.%(ext)s', YOUTUBE_URL],
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE, 
            text=True
        )
    else:
        process = subprocess.Popen(
            ['yt-dlp', '-o', 'Download/%(title)s.%(ext)s', YOUTUBE_URL],
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE,  
            text=True 
        )
    for line in process.stdout:
        yield line
    for line in process.stderr:
        yield line
    process.wait()

    if (do_zip):
        zip_files('video')
    
    yield "影片下載完成"

def download_audio(YOUTUBE_URL, do_zip):
    if(do_zip):
        process = subprocess.Popen(
            ['yt-dlp', '-o', 'Download_TMP/%(title)s.%(ext)s', '--extract-audio', '--audio-format', 'mp3', YOUTUBE_URL],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
    else:
        process = subprocess.Popen(
            ['yt-dlp', '-o', 'Download/%(title)s.%(ext)s', '--extract-audio', '--audio-format', 'mp3', YOUTUBE_URL],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
    for line in process.stdout:
        yield line

    for line in process.stderr:
        yield line

    process.wait()

    if (do_zip):
        zip_files('audio')

    yield "音檔下載完成"

def zip_files(Type):
    zip_folder = 'Download_ZIP'
    if not os.path.exists(zip_folder):
        os.makedirs(zip_folder)
    folder_to_zip = 'Download_TMP'
    zip_filename = f'{zip_folder}/{Type}_{year}_{month}_{day}_{hour}_{minute}_{second}.zip'
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, dirs, files in os.walk(folder_to_zip):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_zip))
    !rm -rf Download_TMP

def gradio_interface(function_choice, YOUTUBE_URL, do_zip):
    if function_choice == "下載影片":
        yield from download_video(YOUTUBE_URL, do_zip)
    elif function_choice == "下載音檔":
        yield from download_audio(YOUTUBE_URL, do_zip)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Radio(["下載影片", "下載音檔"], label="選擇功能"),
        gr.Textbox(label="輸入 YouTube 清單或影片的連結"),
        gr.Checkbox(label="壓縮成 ZIP 檔")
    ],
    outputs="text",
)
iface.queue().launch(share=True)
# 4K 60fps 影片連結(測試用) : https://youtu.be/3aRJtJRa434
# 播放清單 和 影片 不可以放分享連結(share結尾)，要放一般的網址，如 : https://www.youtube.com/playlist?list=PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL